In [43]:
import pybedtools as pbt
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
def clean_axis(ax):
    ax.xaxis.set_tick_params(labelsize=9)
    ax.yaxis.set_tick_params(labelsize=9)
    for i in ['top', 'bottom', 'left', 'right']:
        ax.spines[i].set_visible(False)
    ax.grid(which='major', linestyle='--', alpha=0.5)
    return True

In [10]:
base_dir = '/home/parashar/scratch/quadruplexes/hg19'
chroms = ['chr' + str(x) for x in range(1,23)] + ['chrX', 'chrY']
for i in [5, 10, 15, 20, 25]:
    for j in [0, 3]:
        tot = 0
        num = 0
        for chrom in chroms: 
            fn = '%s/g3_%d_%d_%s_nov.bed' % (base_dir, i, j, chrom)
            with open(fn) as h:
                for l in h:
                    c = l.rstrip('\n').split('\t')
                    tot += int(c[2]) - int(c[1])
                    num += 1
        print (i, j, tot, num)        

5 0 5326617 246128
5 3 123744652 4481756
10 0 17408587 592803
10 3 297726983 8137053
15 0 36911619 989528
15 3 518055494 11363561
20 0 62551418 1395347
20 3 742035308 13777306
25 0 95259462 1814347
25 3 1004005129 15586788


In [15]:
sum([int(x.rstrip('\n').split('\t')[1]) for x in 
    open('/home/parashar/scratch/hg19_resource/hg19.genome').readlines()
         if x.split('\t')[0] in chroms])

3095677412

In [26]:
32079290/(3095677412*2)

0.005181303755302266